In [1]:
import sys, asyncio
from pathlib import Path

if sys.platform == "win32":
    asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())

import chess
import chess.svg
import pandas as pd
from IPython.display import SVG, display

import pyarrow as pa
import pyarrow.parquet as pq
from datetime import datetime

from prompt import generate_data_sample
from chess_explainer import ChessExplainer

In [2]:
engine_path = Path("./stockfish/stockfish-windows-x86-64-avx2.exe")
explainer = ChessExplainer(engine_path=engine_path, depth=18, multipv=15, think_time=0.3)

# Also import our data
df = pd.read_csv('prog_data/train_20k.csv')   # Import prog_data/train_20k.csv instead

In [ ]:
num_samples = 250

# === DATA GENERATION ===
samples = []
for i in range(num_samples):
    print(f"[{i+1:<3}/{num_samples}]")
    df = df.sample(frac=1).reset_index(drop=True)
    fen = df.iloc[0]['FEN']
    board = chess.Board(fen)
    # display(SVG(chess.svg.board(board=board, size=300)))

    # Generate our data and get our train data formatted
    results, final_statement, best_move_uci = explainer.analyze_position(fen, generate_explanation=True)
    explanations = [move['explanation'] for move in results]
    sys, user, model = generate_data_sample(fen, explanations, final_statement, best_move_uci)

    prompt = sys + "\n" + user
    completion = model
    samples.append({'prompt': prompt, 'completion': completion})

# === SAVE TO PARQUET ===
now = datetime.now()
save_filename = f"explainer_clean_{num_samples}_{now.hour:02d}{now.minute:02d}_{now.day:02d}.parquet"
save_path = Path("progdata") / save_filename

# Convert to pyarrow Table and save
table = pa.Table.from_pandas(pd.DataFrame(samples))
save_path.parent.mkdir(exist_ok=True)
pq.write_table(table, save_path)
print(f"Saved {len(samples)} samples to {save_path}")

[1  /250]
[2  /250]
[3  /250]
[4  /250]
[5  /250]
[6  /250]
[7  /250]
[8  /250]
[9  /250]
[10 /250]
[11 /250]
[12 /250]
[13 /250]
[14 /250]
[15 /250]
[16 /250]
[17 /250]
[18 /250]
[19 /250]
[20 /250]
[21 /250]
[22 /250]
[23 /250]
[24 /250]
[25 /250]
[26 /250]
[27 /250]
[28 /250]
[29 /250]
[30 /250]
[31 /250]
[32 /250]
[33 /250]
[34 /250]
[35 /250]
[36 /250]
[37 /250]
[38 /250]
[39 /250]
[40 /250]
[41 /250]
[42 /250]
[43 /250]
[44 /250]
[45 /250]
[46 /250]
[47 /250]
[48 /250]
[49 /250]
[50 /250]
[51 /250]
[52 /250]
[53 /250]
[54 /250]
[55 /250]
[56 /250]
[57 /250]
[58 /250]
[59 /250]
[60 /250]
[61 /250]
[62 /250]
[63 /250]
[64 /250]
[65 /250]
[66 /250]
[67 /250]
[68 /250]
[69 /250]
[70 /250]
[71 /250]
[72 /250]
[73 /250]
[74 /250]
[75 /250]
[76 /250]
[77 /250]
[78 /250]
[79 /250]
[80 /250]
[81 /250]
[82 /250]
[83 /250]
[84 /250]
[85 /250]
[86 /250]
[87 /250]
[88 /250]
[89 /250]


In [4]:
# # Optionally can test various parts of the generation
# df = df.sample(frac=1).reset_index(drop=True)
# fen = df.iloc[0]['board']
# board = chess.Board(fen)
# print(fen)

# display(SVG(chess.svg.board(board=board, size=300)))

# # Analyze postion
# results, final_statement, best_move_uci = explainer.analyze_position(fen, generate_explanation=True)

# for move in results:
#     print(f"{'='*50}\nMove: {move['uci']} ({move['tree'].minimax})\n{'-'*50}")
#     print(move['explanation'])
#     print(f"\n")
#     print(explainer.visualize_tree(move['tree']))
#     print(f"\n")
    
# print(final_statement)

# # strategy = explainer.find_opponent_strategy(analysis[0]["tree"], depth=2)
# # print(strategy)